In [ ]:
!pip install agentpy

In [ ]:
!pip install matplotlib==3.1.3

In [5]:
# Model design
import agentpy as ap
import networkx as nx

# Visualization
import seaborn as sns
import time
from random import randint

In [55]:
class ParkingSpot(ap.Agent):
    def setup(self):
        self.id = 0
        self.carId = 0
        self.isOccupied = False
        self.coordinates = {'x': 0, 'z': 0}

In [56]:
class Car(ap.Agent): 
  
    def setup(self):
        # Parameters
        self.velocity = 10
        self.distance = 0
        self.timeParked = 0
        self.isInStreetTF = 0
        self.isParked = False
        self.assignedParkingSpot = -1
        self.assignedParkingSpotCoor = -1
        self.coordinates = {'x': -39.58, 'y': 14.71, 'z': 43.06}
        self.looking = 0
        self.direction = randint(0, 4)
        self.id = 0
        self.routeEntrance1 = [[19.49, 43.06], [23.7, 37.4], [23.7, 21.9], [20.4, 16.5]]
        self.routeEntrance2 = [[20.4, -7.9], [34.6, -7.9], [34.6, 7.1]]
        self.routeLeaving1 = [[20.4, -7.9], [34.6, -7.9], [34.6, 7.1], [33.2, 19.6], [29.2, 21.1], [29.2, 34.9], [17.9, 45.7]]
        self.routeLeaving2 = [[33.2, 19.6], [29.2, 21.1], [29.2, 34.9], [17.9, 45.7]]
        self.routeEntrance3 = [[28.6, 51.9], [28.6, 67.2], [34.2, 68.8]]
        self.routeEntrance4 = [[34.2, 96.6], [20.1, 96.6]]
        self.routeLeaving3 = [[34.2, 96.6], [20.1, 96.6], [20.1, 69.1], [23.1, 69.1], [23.1, 52.6], [18.2, 47.9]]
        self.routeLeaving4 = [[20.1, 69.1], [23.1, 69.1], [23.1, 52.6], [18.2, 47.9], [-43.5, 48]]
        
  
    def initialize(self, n):
        
        self.id = n
        self.isInStreetTF = 1 # semafaro 1
        self.looking = 1
        self.timeParked = randint(0,11)
        print("Car initialized: ", self.id)
        #asignar cajon
        parkingSpots = self.model.spotsAgents.select(self.model.spotsAgents.isOccupied == False)
        if (len(parkingSpots) == 0):
            
            
            #sacar un auto random
            parkedCars = self.model.cars.select(self.model.cars.isParked == True)
            if(len(parkedCars) != 0):
                car = parkedCars.random()
                parkingSpots2 = self.model.spotsAgents.select(self.model.spotsAgents.idd == car.id)
                parkingSpot2 = parkingSpots2[0]
                
                idd = car.assignedParkingSpot 
                print("Removing car: ", car.id, " --- : ", idd, " type: ", type(car.assignedParkingSpot)) 
                item3 = ap.AttrIter(car.assignedParkingSpot) 
                item = ap.AttrIter([[8]]) 
                print("item: ",item, "  --> ", type(item), "  --> ", type(item3)) 
                item2 = item[0] print("item2: ",item2) item4 = item3[0] print("item4: ",item4) 
                '''for i in range(100): 
                    item = ap.AttrIter([[i]]) 
                    if item == car.assignedParkingSpot: 
                        print("Found at: ", item)'''
                
                self.assignedParkingSpot = parkingSpot2.id
                self.assignedParkingSpotCoor = parkingSpot2.coordinates
                print("Assigned parking spot: ", self.assignedParkingSpot, " : ", parkingSpot2.id)
                car.assignedParkingSpot = -1
                car.assignedParkingSpotCoor = []
                car.isParked = False
                
                #car.goOut()
        else:
            parkingSpot = parkingSpots.random()
            parkingSpot.isOccupied = True
            print(parkingSpot.id)
            self.assignedParkingSpot = parkingSpot.id
            self.assignedParkingSpotCoor = parkingSpot.coordinates
            print("Assigned parking spot: ", self.assignedParkingSpot)
        
        self.move()
        
    def move(self):
        
        #llegar a primer semaforo
        self.looking = 1

        #pasar primer semaforo
        if(self.isInStreetTF == 1 and self.model.trafficLightsAgents.getStatus() != 0):
            self.velocity = 0
        else:
            self.velocity = 10
            
        
        
        #ir a lugar de estacionamiento
        self.getTo(self.assignedParkingSpotCoor)
            
        #estacionarse
        self.isParked = True
        

        #esperar tiempo de estacionamiento
        for i in range(self.timeParked):
            #time.sleep(1)
            pass
        
        #salir del lugar de estacionamiento
        '''self.isParked = False
        self.looking = 0
        parkingSpots = self.model.spotsAgents.select(self.model.spotsAgents.id == self.assignedParkingSpot)
        parkingSpot = parkingSpots.random()
        parkingSpot.isOccupied = False
        self.assignedParkingSpot = -1'''
        
        
        #ir a segundo semaforo
        if (self.assignedParkingSpot < 19):
            self.isInStreetTF = 2
            #self.getTo(coordenadas del semáforo 2)
        else:
            self.isInStreetTF = 3
            #self.getTo(coordenadas del semáforo 3)


        #salir del juego
        #self.goOut()
        
    def getTo(self, coordinates):
        #calcular ruta para llegar a las coordenadas proporcionadas
        pass

    def goOut(self):
        #calcular ruta para salir
        
        self.isParked = False
        self.looking = 0
        '''parkingSpots = self.model.spotsAgents.select(self.model.spotsAgents.id == self.assignedParkingSpot)
        parkingSpot = parkingSpots.random()
        parkingSpot.isOccupied = False'''
        self.assignedParkingSpot = -1
        pass
    
    

In [57]:
# Define parameter ranges
class TrafficLight(ap.Agent):
    
    def setup(self):
        self.state = 0 # Green
        self.id = 0
  
    def changeStatus(self):
        if self.state == 0:
            self.state = 1 # Yellow
        elif self.state == 1:
            self.state = 2 # Red
        else:
            self.state = 0
    
    def getStatus(self):
        return self.state

    def start(self):
        print("starting semaforo: ", self.id)
        pass

In [58]:
class ParkingLotModel(ap.Model):

    def setup(self):
    
        self.availableSpots = self.p.parkingAgents        
        self.parkedCars = 0
        self.steps = 0
        self.cars = ap.AgentList(self, self.p.n, Car)
        
        cajones = [[-5.574306, 75.91479], [-5.574306, 78.38479], [-5.574306, 80.85479], [-5.574306, 83.36479], 
                   [-5.574306, 85.83479], [-5.574306, 88.25479], [-5.574306, 90.67479], [-5.574306, 93.14479],
                   [-5.574306, 95.74479], [-2.404306, 95.74479], [-2.404306, 93.27479], [-2.404306, 90.75479],
                   [-2.404306, 88.25479], [-2.404306, 85.85279], [-2.404306, 83.33479], [-2.404306, 80.89479],
                   [-2.404306, 78.50479], [-2.404306, 75.93479], [-2.654306, 15.72479], [-2.654306, 13.17479],
                   [-2.654306, 10.73479], [-2.654306, 8.234787], [-2.654306, 5.774787], [-2.654306, 3.384787],
                   [-2.654306, 0.8547869], [-2.654306, -1.585213], [-2.654306, -4.125213], [-5.864305, -4.125213],
                   [-5.864305, -1.655213], [-5.864305, 0.7447869], [-5.864305, 3.304787], [-5.864305, 5.654787],
                   [-5.864305, 8.254787], [-5.864305, 10.66479], [-5.864305, 13.10479], [-5.864305, 15.65479]]
        
        self.spotsAgents = ap.AgentList(self, self.p.parkingAgents, ParkingSpot)
        count = 0
        for spot in self.spotsAgents:
            spot.id = count + 1
            spot.coordinates['x'] = cajones[count][0]
            spot.coordinates['z'] = cajones[count][1]
            count += 1
            
        self.trafficLightsAgents = ap.AgentList(self, 3, TrafficLight)
        count = 1
        for trafficLight in self.trafficLightsAgents:
            trafficLight.id = count
            count += 1
        
        self.trafficLightsAgents[0].start()
        self.trafficLightsAgents[1].start()
        self.trafficLightsAgents[2].start()

    def step(self):
        #initialize car
        for i in range(self.p.vehicleAgents):
            car = self.cars.random()
            car.initialize(i)
            #print(self.steps)
            #self.cars[self.steps].initialize(self.steps)
            self.steps += 1

    def update(self):
        self.record('parkedCars', self.getParkedCars())
        self.record('steps', self.steps)
        
    def getParkedCars(self):
        return len(self.cars.select(self.cars.isParked == True))

In [59]:
# Define parameter ranges
parameter_ranges = {
    'vehicleAgents' : 100,
    'parkingAgents': 36,
    'steps': 100,  # Number of simulation steps
    'speed': 0.05,  # Speed of connections per step
    'n': ap.Values(100,1000)  # Number of agents
}

model = ParkingLotModel(parameter_ranges)
results = model.run()

# Create sample for different values of n
#sample = ap.Sample(parameter_ranges)

# Keep dynamic variables
#exp = ap.Experiment(ParkingLotModel, sample, iterations=25, record=True) 

# Perform 75 separate simulations (3 parameter combinations * 25 repetitions)
#results = exp.run()
results.save()
sns.set_theme()
results = ap.DataDict.load('ParkingLotModel')
sns.lineplot(
  data=results.arrange_variables(),
  x='steps',
  y='parkedCars'
)


starting semaforo:  1
starting semaforo:  2
starting semaforo:  3
Car initialized:  0
[16]
Assigned parking spot:  [16]
Car initialized:  1
[19]
Assigned parking spot:  [19]
Car initialized:  2
[33]
Assigned parking spot:  [33]
Car initialized:  3
[21]
Assigned parking spot:  [21]
Car initialized:  4
[32]
Assigned parking spot:  [32]
Car initialized:  5
[5]
Assigned parking spot:  [5]
Car initialized:  6
[10]
Assigned parking spot:  [10]
Car initialized:  7
[7]
Assigned parking spot:  [7]
Car initialized:  8
[15]
Assigned parking spot:  [15]
Car initialized:  9
[36]
Assigned parking spot:  [36]
Car initialized:  10
[30]
Assigned parking spot:  [30]
Car initialized:  11
[14]
Assigned parking spot:  [14]
Car initialized:  12
[11]
Assigned parking spot:  [11]
Car initialized:  13
[17]
Assigned parking spot:  [17]
Car initialized:  14
[6]
Assigned parking spot:  [6]
Car initialized:  15
[13]
Assigned parking spot:  [13]
Car initialized:  16
[8]
Assigned parking spot:  [8]
Car initialized: 

AgentpyError: AgentIter has to be converted to list for item lookup.